In [2]:
import requests
import json
import pandas as pd
from datetime import datetime

In [3]:
# Get all bike points from TfL API
app_id = "f49aabef"
app_key = "740a84c9b887a6ae4de325a3d32ac795"
params = {'app_id': app_id, 'app_key': app_key}
r = requests.get('https://api.tfl.gov.uk/AirQuality', params=params)
air_quality = r.json()

In [4]:
# Today's forecast
air_quality['currentForecast'][0]

{u'$id': u'2',
 u'$type': u'Tfl.Api.Presentation.Entities.CurrentForecast, Tfl.Api.Presentation.Entities',
 u'forecastBand': u'Low',
 u'forecastID': u'12420',
 u'forecastSummary': u'Low air pollution forecast valid from Friday 16 June to end of Friday 16 June GMT',
 u'forecastText': u'Dry and warm weather conditions are forecast for Friday. However, greater cloud cover and slightly lower temperatures than in recent days will limit the potential for the photochemical generation of ground-level ozone. A steady breeze from the west should effectively disperse locally-emitted pollutants.&lt;br/&gt;&lt;br/&gt;Pollution is expected to remain &#39;Low&#39; for the following pollutants during the forecast period:&lt;br/&gt;&lt;br/&gt;Ozone&lt;br/&gt;Nitrogen Dioxide&lt;br/&gt;PM10 particulates&lt;br/&gt;PM25 particulates&lt;br/&gt;Sulphur Dioxide&lt;br/&gt;&lt;br/&gt;&lt;br/&gt;&lt;br/&gt;&lt;br/&gt;&lt;br/&gt;&lt;br/&gt;&lt;br/&gt;',
 u'forecastType': u'Current',
 u'nO2Band': u'Low',
 u'o3Ban

In [5]:
# Tomorrow's forecast
air_quality['currentForecast'][1]

{u'$id': u'3',
 u'$type': u'Tfl.Api.Presentation.Entities.CurrentForecast, Tfl.Api.Presentation.Entities',
 u'forecastBand': u'Moderate',
 u'forecastID': u'12421',
 u'forecastSummary': u'Moderate ozone air pollution forecast valid from Saturday 17 June to end of Monday 19 June GMT',
 u'forecastText': u'Saturday will be dry and sunny for most of the day, feeling warm with periods of cloud cover. A relatively clean air feed from the Atlantic should help dispersion of local emissions and pollution is likely to remain &#39;Low&#39;.&lt;br/&gt;&lt;br/&gt;A fine settle conditions forecast for Sunday and Monday with sunshine for  both days. Very light westerly winds will start to arrive from the east by Sunday evening, importing pollution from the north of the Continent. This air feed, combined with hours of unbroken sunshine will allow ozone production to build through both days, with &#39;Moderate&#39; ozone likely in the afternoons, particularly in suburban and outer-London locations.&lt;b

In [6]:
import math

def distance(origin, destination):
    lat1, lon1 = origin
    lat2, lon2 = destination
    radius = 6371 # km

    dlat = math.radians(lat2-lat1)
    dlon = math.radians(lon2-lon1)
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = radius * c

    return d

In [7]:
origin = (51.5320521, -0.1092356)
destination = (51.529163, -0.10997)
distance(origin, destination)

0.3252451958196465

## Get bike point data

In [8]:
r = requests.get('https://api.tfl.gov.uk/BikePoint', params=params)
bike_points = r.json()

In [12]:
res = []
for point in bike_points:
    row = {}
    for e in point['additionalProperties']:
        if e['key'] == 'NbDocks':
            row['docks'] = int(e['value'])
    row['name'] = point['commonName']
    row['id'] = point['id']
    row['lat'] = point['lat']
    row['lon'] = point['lon']
    res.append(row)
res_df = pd.DataFrame(res)

In [13]:
res

[{'docks': 19,
  'id': u'BikePoints_1',
  'lat': 51.529163,
  'lon': -0.10997,
  'name': u'River Street , Clerkenwell'},
 {'docks': 37,
  'id': u'BikePoints_2',
  'lat': 51.499606,
  'lon': -0.197574,
  'name': u'Phillimore Gardens, Kensington'},
 {'docks': 32,
  'id': u'BikePoints_3',
  'lat': 51.521283,
  'lon': -0.084605,
  'name': u'Christopher Street, Liverpool Street'},
 {'docks': 23,
  'id': u'BikePoints_4',
  'lat': 51.530059,
  'lon': -0.120973,
  'name': u"St. Chad's Street, King's Cross"},
 {'docks': 27,
  'id': u'BikePoints_5',
  'lat': 51.49313,
  'lon': -0.156876,
  'name': u'Sedding Street, Sloane Square'},
 {'docks': 18,
  'id': u'BikePoints_6',
  'lat': 51.518117,
  'lon': -0.144228,
  'name': u'Broadcasting House, Marylebone'},
 {'docks': 16,
  'id': u'BikePoints_7',
  'lat': 51.5343,
  'lon': -0.168074,
  'name': u"Charlbert Street, St. John's Wood"},
 {'docks': 18,
  'id': u'BikePoints_8',
  'lat': 51.528341,
  'lon': -0.170134,
  'name': u"Lodge Road, St. John's Wo

In [15]:
data = {'objects': res}
with open('TfL_bike_station_data.json', 'wb') as outfile:
    json.dump(data, outfile)

In [22]:
distances = []
for row in data['objects']:
    entry = {}
    entry[distance(origin, (row['lat'], row['lon']))] = row['id']
    distances.append(entry)

In [26]:
distances = sorted(distances)

In [35]:
distances

[{0.0636024239073157: u'BikePoints_189'},
 {0.22548810812822379: u'BikePoints_339'},
 {0.259749212051012: u'BikePoints_254'},
 {0.2602941423619337: u'BikePoints_75'},
 {0.26236883863647087: u'BikePoints_93'},
 {0.31371285509247643: u'BikePoints_234'},
 {0.3252451958196465: u'BikePoints_1'},
 {0.47505819772117075: u'BikePoints_170'},
 {0.5156979045969995: u'BikePoints_123'},
 {0.5394422831225835: u'BikePoints_697'},
 {0.5472920943301667: u'BikePoints_70'},
 {0.5552151558637941: u'BikePoints_227'},
 {0.6128976324544358: u'BikePoints_674'},
 {0.6237050595203304: u'BikePoints_264'},
 {0.6437234263376359: u'BikePoints_326'},
 {0.6454147804552433: u'BikePoints_204'},
 {0.6553597416326472: u'BikePoints_365'},
 {0.6578964516632273: u'BikePoints_695'},
 {0.6813735556482937: u'BikePoints_439'},
 {0.6845392686340649: u'BikePoints_51'},
 {0.7074953749582851: u'BikePoints_593'},
 {0.7082517956433935: u'BikePoints_344'},
 {0.8164591118695953: u'BikePoints_544'},
 {0.8211632656495157: u'BikePoints_26

In [36]:
distances[0]

{0.0636024239073157: u'BikePoints_189'}

In [34]:
for e in distances[0:3]:
    for key, value in e.items():
        print value

BikePoints_189
BikePoints_339
BikePoints_254
